# Implementing a Route Planner
In this project you will use A\* search to implement a "Google-maps" style route planning algorithm.

In [1]:
# Run this cell first!
#%matplotlib notebook
from helpers import Map, load_map, show_map
from student_code import shortest_path
#%reload_ext autoreload
%load_ext autoreload
%autoreload 2

In [2]:
path_goal_min_val = float('inf')
path_goal_min_val
 

inf

### Map Basics

In [3]:
map_10 = load_map('map-10.pickle')
show_map(map_10)

The map above (run the code cell if you don't see it) shows a disconnected network of 10 intersections. The two intersections on the left are connected to each other but they are not connected to the rest of the road network. On the graph above, the edge between 2 nodes(intersections) represents a literal straight road not just an abstract connection of 2 cities.

These `Map` objects have two properties you will want to use to implement A\* search: `intersections` and `roads`

**Intersections**

The `intersections` are represented as a dictionary. 

In this example, there are 10 intersections, each identified by an x,y coordinate. The coordinates are listed below. You can hover over each dot in the map above to see the intersection number.

In [69]:
map_10.intersections

{0: [0.7798606835438107, 0.6922727646627362],
 1: [0.7647837074641568, 0.3252670836724646],
 2: [0.7155217893995438, 0.20026498027300055],
 3: [0.7076566826610747, 0.3278339270610988],
 4: [0.8325506249953353, 0.02310946309985762],
 5: [0.49016747075266875, 0.5464878695400415],
 6: [0.8820353070895344, 0.6791919587749445],
 7: [0.46247219371675075, 0.6258061621642713],
 8: [0.11622158839385677, 0.11236327488812581],
 9: [0.1285377678230034, 0.3285840695698353]}

**Roads**

The `roads` property is a list where, if `i` is an intersection, `roads[i]` contains a list of the intersections that intersection `i` connects to.

In [70]:
# this shows that intersection 0 connects to intersections 7, 6, and 5
map_10.roads[0]

[7, 6, 5]

In [4]:
# This shows the full connectivity of the map
map_10.roads

[[7, 6, 5],
 [4, 3, 2],
 [4, 3, 1],
 [5, 4, 1, 2],
 [1, 2, 3],
 [7, 0, 3],
 [0],
 [0, 5],
 [9],
 [8]]

In [4]:
# map_40 is a bigger map than map_10
map_40 = load_map('map-40.pickle')
show_map(map_40)

In [5]:
map_40.roads

[[36, 34, 31, 28, 17],
 [35, 31, 27, 26, 25, 20, 18, 17, 15, 6],
 [39, 36, 21, 19, 9, 7, 4],
 [35, 20, 15, 11, 6],
 [39, 36, 21, 19, 9, 7, 2],
 [32, 16, 14],
 [35, 20, 15, 11, 1, 3],
 [39, 36, 22, 21, 19, 9, 2, 4],
 [33, 30, 14],
 [36, 21, 19, 2, 4, 7],
 [31, 27, 26, 25, 24, 18, 17, 13],
 [35, 20, 15, 3, 6],
 [37, 34, 31, 28, 22, 17],
 [27, 24, 18, 10],
 [33, 30, 16, 5, 8],
 [35, 31, 26, 25, 20, 17, 1, 3, 6, 11],
 [37, 30, 5, 14],
 [34, 31, 28, 26, 25, 18, 0, 1, 10, 12, 15],
 [31, 27, 26, 25, 24, 1, 10, 13, 17],
 [21, 2, 4, 7, 9],
 [35, 26, 1, 3, 6, 11, 15],
 [2, 4, 7, 9, 19],
 [39, 37, 29, 7, 12],
 [38, 32, 29],
 [27, 10, 13, 18],
 [34, 31, 27, 26, 1, 10, 15, 17, 18],
 [34, 31, 27, 1, 10, 15, 17, 18, 20, 25],
 [31, 1, 10, 13, 18, 24, 25, 26],
 [39, 36, 34, 31, 0, 12, 17],
 [38, 37, 32, 22, 23],
 [33, 8, 14, 16],
 [34, 0, 1, 10, 12, 15, 17, 18, 25, 26, 27, 28],
 [38, 5, 23, 29],
 [8, 14, 30],
 [0, 12, 17, 25, 26, 28, 31],
 [1, 3, 6, 11, 15, 20],
 [39, 0, 2, 4, 7, 9, 28],
 [12, 16, 22, 

In [59]:
map_40.intersections

{0: [0.7801603911549438, 0.49474860768712914],
 1: [0.5249831588690298, 0.14953665513987202],
 2: [0.8085335344099086, 0.7696330846542071],
 3: [0.2599134798656856, 0.14485659826020547],
 4: [0.7353838928272886, 0.8089961609345658],
 5: [0.09088671576431506, 0.7222846879290787],
 6: [0.313999018186756, 0.01876171413125327],
 7: [0.6824813442515916, 0.8016111783687677],
 8: [0.20128789391122526, 0.43196344222361227],
 9: [0.8551947714242674, 0.9011339078096633],
 10: [0.7581736589784409, 0.24026772497187532],
 11: [0.25311953895059136, 0.10321622277398101],
 12: [0.4813859169876731, 0.5006237737207431],
 13: [0.9112422509614865, 0.1839028760606296],
 14: [0.04580558670435442, 0.5886703168399895],
 15: [0.4582523173083307, 0.1735506267461867],
 16: [0.12939557977525573, 0.690016328140396],
 17: [0.607698913404794, 0.362322730884702],
 18: [0.719569201584275, 0.13985272363426526],
 19: [0.8860336256842246, 0.891868301175821],
 20: [0.4238357358399233, 0.026771817842421997],
 21: [0.825249

### Advanced Visualizations

The map above shows a network of roads which spans 40 different intersections (labeled 0 through 39). 

The `show_map` function which generated this map also takes a few optional parameters which might be useful for visualizaing the output of the search algorithm you will write.

* `start` - The "start" node for the search algorithm.
* `goal`  - The "goal" node.
* `path`  - An array of integers which corresponds to a valid sequence of intersection visits on the map.

In [6]:
# run this code, note the effect of including the optional
# parameters in the function call.
show_map(map_40, start=5, goal=34, path=[5,16,37,12,34])

### Writing your algorithm
You should open the file `student_code.py` in another tab and work on your algorithm there. Do that by selecting `File > Open` and then selecting the appropriate file.

The algorithm you write will be responsible for generating a `path` like the one passed into `show_map` above. In fact, when called with the same map, start and goal, as above you algorithm should produce the path `[5, 16, 37, 12, 34]`

```bash
> shortest_path(map_40, 5, 34)
[5, 16, 37, 12, 34]
```

In [61]:
path = shortest_path(map_40, 5, 34)
if path == [5, 16, 37, 12, 34]:
    print("great! Your code works for these inputs!")
else:
    print("something is off, your code produced the following:")
    print(path)

great! Your code works for these inputs!


### Testing your Code
If the code below produces no errors, your algorithm is behaving correctly. You are almost ready to submit! Before you submit, go through the following submission checklist:

**Submission Checklist**

1. Does my code pass all tests?
2. Does my code implement `A*` search and not some other search algorithm?
3. Do I use an **admissible heuristic** to direct search efforts towards the goal?
4. Do I use data structures which avoid unnecessarily slow lookups?

When you can answer "yes" to all of these questions, submit by pressing the Submit button in the lower right!

In [62]:
from test import test

test(shortest_path)

All tests pass! Congratulations!


In [17]:
import heapq
from math import pow, sqrt
from collections import namedtuple

Cost = namedtuple('Cost', ['total', 'journey', 'to_goal'])  # (float, float)
Path = namedtuple('Path', ['cost', 'intersections', 'previous', 'frontier'])  # (Cost, [int, int ...], int, int)


def euclidean_distance(origin_point: [float, float], destination_point: [float, float]) -> float:
    """
    Given two points returns their euclidean distance
    :param origin_point: origin point, in the 2D cartesian space
    :param destination_point: destination point, in the 2D cartesian space
    :return: euclidean distance between the two points
    """

    return sqrt(pow((origin_point[0] - destination_point[0]), 2) + pow((origin_point[1] - destination_point[1]), 2))


def estimated_distance(path_frontier_point: [float, float], goal_point: [float, float]) -> float:
    """
    Estimates the distance between the current path frontier point and the goal. Accomplished the A* optimization
    requirement of having an estimating function that underestimates. As in a 2D-Cartesian space, the straight line is
    always the smallest possible distance between two points; guaranteeing the "underestimation" requirement
    :param path_frontier_point: path frontier point
    :param goal_point: goal point
    :return: estimated euclidean distance
    """
    return euclidean_distance(origin_point=path_frontier_point, destination_point=goal_point)


def update_path(map: object, path: Path, new_frontier: int, goal: int) -> Path:
    """
    Given a path and a next point, updates the path
    :param map: map of the current 2D space
    :param path: current traversed path
    :param new_frontier: coordinates of next point to add to the path
    :param goal: coordinates of the goal intersection
    :return: path costs and intersections updated with new point
    """
    traversed_distance = euclidean_distance(origin_point=map.intersections[path.frontier],
                                            destination_point=map.intersections[new_frontier])
    new_path_cost_journey = path.cost.journey + traversed_distance
    new_path_cost_to_goal = estimated_distance(path_frontier_point=map.intersections[new_frontier],
                                               goal_point=map.intersections[goal])
    new_path_cost_total = new_path_cost_journey + new_path_cost_to_goal

    new_path_intersections = path.intersections + [new_frontier]

    new_path = Path(Cost(new_path_cost_total, new_path_cost_journey, new_path_cost_to_goal),
                    new_path_intersections, path.frontier, new_frontier)

    return new_path


def shortest_path(map: object, start: int, goal: int) -> list:
    """
    Given a map and a start and end point, returns the shortest path, based on A* algorithm
    :param map: map of the current 2D space
    :param start: coordinates of the original intersection
    :param goal: coordinates of the goal intersection
    :return:
    """
    paths = list()
    path_goal_min_val = float('inf')
    path_goal_min = None
    
    # Check if already in goal
    if start == goal:
        return [start]

    # Initialize paths
    goal_initial_distance = estimated_distance(path_frontier_point=map.intersections[start],
                                               goal_point=map.intersections[goal])
    print('goal_initial_distance',goal_initial_distance)
    path = Path(Cost(goal_initial_distance, 0, goal_initial_distance), [start], start, start)
    
    print('path-->',path)
    heapq.heappush(paths, path)

    while len(paths) >= 1:
        nearest_frontier_path = heapq.heappop(paths)
        print('nearest_frontier_path-->',nearest_frontier_path.frontier)
        for neighbor_road in map.roads[nearest_frontier_path.frontier]:

            if neighbor_road == nearest_frontier_path.previous:  # Avoid returning to backwards
                continue
            else:  # Continue

                new_path = update_path(map=map, path=nearest_frontier_path, new_frontier=neighbor_road, goal=goal)

                if neighbor_road == goal:  # Reached destination with a path
                    if new_path.cost.total < path_goal_min_val:  # Better than previous path
                        path_goal_min_val = new_path.cost.total
                        path_goal_min = new_path.intersections
                    else:  # Reached destination, with higher cost -> disregard
                        pass
                else:
                    if path_goal_min is not None:  # Already found the goal with a path
                        if new_path.cost.total >= path_goal_min_val:  # Path not reached goal and already costly
                            pass
                        else:  # Cheaper path, keep exploring
                            heapq.heappush(paths, new_path)
                    else:  # Not yet found the goal, keep exploring
                        heapq.heappush(paths, new_path)

    if path_goal_min is not None:
        return path_goal_min
    else:
        return -1


In [18]:
path = shortest_path(map_40, 5, 34)
print(path)

goal_initial_distance 0.574734871592805
path--> Path(cost=Cost(total=0.574734871592805, journey=0, to_goal=0.574734871592805), intersections=[5], previous=5, frontier=5)
nearest_frontier_path--> 5
nearest_frontier_path--> 16
nearest_frontier_path--> 37
nearest_frontier_path--> 12
nearest_frontier_path--> 14
nearest_frontier_path--> 14
nearest_frontier_path--> 22
nearest_frontier_path--> 30
nearest_frontier_path--> 32
nearest_frontier_path--> 29
[5, 16, 37, 12, 34]


In [14]:
v = sqrt(pow((0.09 - 0.58), 2) + pow((0.722 - 0.427), 2))
print(v)


0.5719484242482009
